# Hands-On Pertemuan 3: Implementasi MapReduce Sederhana

## Tujuan:
- Memahami konsep MapReduce dalam distribusi data besar.
- Mengimplementasikan algoritma sederhana menggunakan MapReduce pada Hadoop.
- Melakukan analisis hasil MapReduce pada dataset.

### 1. Pengenalan MapReduce
MapReduce adalah model pemrograman yang digunakan untuk pemrosesan data besar secara paralel di beberapa node dalam kluster Hadoop.
- **Map**: Fase pertama di mana data dipecah menjadi unit-unit kecil (key-value pairs).
- **Reduce**: Fase kedua di mana hasil dari fase Map dikombinasikan untuk menghasilkan output yang lebih kecil.

- **Tugas 1**: Pelajari bagaimana MapReduce bekerja dengan dataset sederhana dan coba implementasikan konsep key-value pair.

### 2. Implementasi Sederhana: Word Count
Algoritma Word Count adalah salah satu contoh sederhana dari MapReduce. Dalam tugas ini, kita akan menghitung jumlah kata dalam dataset.

1. **Map Function**: Fungsi yang memecah teks menjadi kata-kata individual.
   ```python
   def map_function(text):
       for word in text.split():
           yield (word, 1)
   ```
2. **Reduce Function**: Fungsi yang menggabungkan hasil dari fase Map untuk menghitung frekuensi kata.
   ```python
   from collections import defaultdict

   def reduce_function(pairs):
       result = defaultdict(int)
       for word, count in pairs:
           result[word] += count
       return result
   ```
- **Tugas 2**: Implementasikan fungsi `map_function` dan `reduce_function` pada dataset teks sederhana, lalu hitung jumlah kata.

In [ ]:
from collections import defaultdict

# Map function: Splits text into words and pairs each word with the number 1
def map_function(text):
    for word in text.split():
        yield (word, 1)

# Reduce function: Combines the counts of each word
def reduce_function(pairs):
    result = defaultdict(int)
    for word, count in pairs:
        result[word] += count
    return result

# Example dataset
dataset = [
    "ini adalah contoh",
    "ini adalah contoh lain",
    "ini adalah contoh yang lebih kompleks"
]

# Applying the MapReduce logic
# Step 1: Apply map function to each text in the dataset
mapped = []
for text in dataset:
    mapped.extend(map_function(text))

# Step 2: Apply reduce function to aggregate word cosunts
reduced = reduce_function(mapped)

# Display the result
print(reduced)


defaultdict(<class 'int'>, {'ini': 3, 'adalah': 3, 'contoh': 3, 'lain': 1, 'yang': 1, 'lebih': 1, 'kompleks': 1})


### 3. Menjalankan Word Count di Hadoop
1. Siapkan file teks yang ingin dihitung jumlah katanya.
   ```bash
   hdfs dfs -put input.txt /user/student/input/
   ```
2. Jalankan perintah MapReduce pada file tersebut:
   ```bash
   hadoop jar $HADOOP_HOME/share/hadoop/mapreduce/hadoop-mapreduce-examples-*.jar wordcount /user/student/input/ /user/student/output/
   ```
3. Tampilkan hasilnya:
   ```bash
   hdfs dfs -cat /user/student/output/part-r-00000
   ```
- **Tugas 3**: Upload file teks ke HDFS, jalankan perintah MapReduce Word Count, dan tampilkan hasilnya.

### 4. Menganalisis Dataset Besar
Untuk memperdalam pemahaman, kita akan menjalankan algoritma MapReduce pada dataset yang lebih besar.
- Pilih dataset besar yang dapat diunduh dari [Kaggle](https://www.kaggle.com/) atau sumber lain.
- Jalankan Word Count pada dataset besar tersebut dan analisis hasilnya.

- **Tugas 4**: Cari dataset besar, jalankan MapReduce untuk menghitung kata, dan buat laporan analisis hasil.

In [ ]:
from collections import defaultdict

# Map function: Splits text into words and pairs each word with the number 1
def map_function(text):
    for word in text.split():
        yield (word, 1)

# Reduce function: Combines the counts of each word
def reduce_function(pairs):
    result = defaultdict(int)
    for word, count in pairs:
        result[word] += count
    return result

# Example dataset
dataset = [
    "ini adalah contoh",
    "ini adalah contoh lain",
    "ini adalah contoh yang lebih kompleks"
]

# Applying the MapReduce logic
# Step 1: Apply map function to each text in the dataset
mapped = []
for text in dataset:
    mapped.extend(map_function(text))

# Step 2: Apply reduce function to aggregate word counts
reduced = reduce_function(mapped)

# Display the result
print(reduced)

### 5. Tugas Tambahan: Custom MapReduce Algorithm
- Buat algoritma MapReduce lainnya, seperti menghitung rata-rata nilai, atau menghitung frekuensi kemunculan elemen tertentu di dalam dataset.

In [ ]:
from mrjob.job import MRJob

class MRCountOpen(MRJob):

    def configure_args(self):
        super(MRCountOpen, self).configure_args()
        self.add_passthru_arg('--month', type=int, default=6, help='Month range for calculation in months')

    def mapper(self, _, line):
        # Data dalam format: timestamp,open,high,low,close,volume
        fields = line.split(',')
        try:
            timestamp = float(fields[0])
            open_price = float(fields[1])
            end_time = 1726538400  # Waktu akhir (last timestamp)
            month = self.options.month
            start_time = end_time - (month * 30 * 24 * 3600)  # Hitung waktu mulai

            # Jika timestamp dalam rentang waktu
            if start_time <= timestamp <= end_time:
                yield open_price, 1  # Keluarkan harga open dan '1' untuk penghitungan
        except ValueError:
            pass  # Jika ada nilai yang tidak bisa di-parse, lewati baris ini

    def reducer(self, open_price, counts):
        # Hitung berapa kali harga "open" yang sama muncul
        yield open_price, sum(counts)

if __name__ == '__main__':
    MRCountOpen.run()